In [13]:
from datetime import datetime, timedelta
import pandas as pd
import xgboost as xgb
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np
from sklearn.model_selection import cross_val_predict

from app.services.dataframe_service import get_technical_data_as_dataframe, get_data_as_dataframe

In [14]:
today = datetime.today().strftime("%Y-%m-%d")

top_50_stock = ["AEFES", "AKBNK", "AKSA", "AKSEN", "ALARK", "ARCLK", "ASELS", "BERA", "BIMAS", "DOHOL",
                "EGEEN", "EKGYO", "ENJSA", "ENKAI", "EREGL", "FROTO", "GARAN", "GESAN", "GUBRF",
                "HALKB", "HEKTS", "ISCTR", "ISGYO", "KCHOL", "KONTR", "KORDS", "KOZAA", "KOZAA",
                "KOZAL", "KRDMD", "MGROS", "ODAS", "OYAKC", "PETKM", "PGSUS", "SAHOL", "SASA",
                "SISE", "SMRTG", "SOKM", "TAVHL", "TCELL", "THYAO", "TKFEN", "TOASO", "TSKB",
                "TTKOM", "TUPRS", "VAKBN", "VESTL", "YKBNK"]

top_1_stock = ["THYAO"]

for stock_symbol in top_1_stock:
    data_technical_info = get_technical_data_as_dataframe(schema_name="technical", table_name=stock_symbol)
    data_basic_info = get_data_as_dataframe(schema_name="public", table_name=stock_symbol)

In [15]:
data_df = data_technical_info.merge(data_basic_info, on="date", how="left")
data_df["percentage"] = data_df["percentage"].shift(-1)
data_df = data_df.dropna()

In [16]:
selected_columns = ["date", "RSI_14", "STOCHk_9_6_3", "STOCHd_9_6_3", "stoch_diff", "STOCHRSIk_14_14_3_3", "MACD_12_26_9", "MACDh_12_26_9",
                    "MACDs_12_26_9", "ADX_14", "DMP_14", "DMN_14", "WILLR_14", "CCI_14_0.015", "ATRr_14", "hl_ratio_14",
                    "UO_7_14_28", "ROC_14", "Bull_power_13", "Bear_power_13", "SMA_interpretation_5", "SMA_interpretation_10",
                    "SMA_interpretation_20", "SMA_interpretation_50",
                    "EMA_interpretation_5", "EMA_interpretation_10", "EMA_interpretation_20", "EMA_interpretation_50",
                    "classic_pivot_interpretation", "fibonacci_pivot_interpretation",
                    "camarilla_pivot_interpretation", "woodie_pivot_interpretation", "demark_pivot_interpretation", "percentage"]

data = data_df[selected_columns]


In [17]:
interpretation_columns = [col for col in data.columns if 'interpretation' in col]
for column in interpretation_columns:
    dummies = pd.get_dummies(data[column], prefix=column)
    data = pd.concat([data, dummies], axis=1)
    data.drop(column, axis=1, inplace=True)

In [18]:
def model_evaluation(models, X, y):
    for i, v in models:
        #prediction = cross_val_predict(v, X, y, cv=2)
        prediction = v.predict(X)
        r2 = r2_score(y, prediction)
        n = X.shape[0]  # örnek sayısı
        p = X.shape[1]  # özellik sayısı
        adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

        print(f"----------------------------- {i} Model Evaluation -----------------------------")
        print("R-Kare değeri                     : {}".format(r2))
        print("Adj. R-Kare değeri                : {}".format(adjusted_r2))
        print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y, prediction)))
        print("Ortalama Kare Hata (MSE)          : {}".format(mse(y, prediction)))
        print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y, prediction)))

In [19]:
y = data["percentage"]
X = data.drop(["date", "percentage"], axis=1)

X_train, X_test, y_train, y_test = X.iloc[:-100, :], X.iloc[-100:, :], y[:-100], y[-100:]

xgb_clf = xgb.XGBRegressor()
xgb_clf.fit(X_train, y_train)
models = []
models.append(('XGBOOST', xgb_clf))
model_evaluation(models, X_test, y_test)

predictions = xgb_clf.predict(X)
data["predictions"] = predictions

----------------------------- XGBOOST Model Evaluation -----------------------------
R-Kare değeri                     : -0.6225246295026587
Adj. R-Kare değeri                : -4.0196855725238505
Ortalama Mutlak Hata (MAE)        : 3.5155982272546296
Ortalama Kare Hata (MSE)          : 22.40130450206773
Kök Ortalama Kare Hata (RMSE)     : 4.73300163765741


In [20]:
def strategy(data):
    BUDGET = 10000
    stocks = 0
    # İşaret değişimlerini ve ilgili tarihleri saklamak için boş bir liste oluşturma
    sign_changes = []
    data = data.reset_index()
    # İlk satırdan başlayarak her satırı kontrol etme
    for i in range(1, len(data)):
        try:
            current_pred = data.loc[i, 'predictions']
            previous_pred = data.loc[i - 1, 'predictions']
            
            # İşaret değişimi kontrolü
            if (current_pred > 0 and previous_pred < 0) or (current_pred < 0 and previous_pred > 0):
                date = data.loc[i, 'date']
                change_type = 'Negatiften Pozitife' if current_pred > 0 else 'Pozitiften Negatife'
                real_date = date + timedelta(days=1)
                open_value = data.loc[i+1, 'open']

                if change_type == 'Negatiften Pozitife':
                    stocks_to_buy = BUDGET // open_value
                    stocks += stocks_to_buy
                    BUDGET -= stocks_to_buy * open_value

                elif change_type == 'Pozitiften Negatife':
                    BUDGET += stocks * open_value
                    stocks = 0

                sign_changes.append((date, change_type, real_date, open_value, BUDGET, stocks))
        except:
            pass

    # İşaret değişimlerini ve ilgili tarihleri yazdırma
    for change in sign_changes:
        print(f"Tarih: {change[0]}, İşaret Değişimi: {change[1]}, Alınması yada satılması gereken tarih(Sabah): {change[2]}, Açılış Değeri: {change[3]}, Bütçe: {change[4]}, Hisseler: {change[5]}")

    return sign_changes[-1][4] + sign_changes[-1][5] * sign_changes[-1][3]


predictions = xgb_clf.predict(X)
data_df["predictions"] = predictions

BUDGET = strategy(data_df.iloc[-100:, :])
print(BUDGET)


Tarih: 2022-12-09, İşaret Değişimi: Pozitiften Negatife, Alınması yada satılması gereken tarih(Sabah): 2022-12-10, Açılış Değeri: 142.5, Bütçe: 10000.0, Hisseler: 0
Tarih: 2022-12-13, İşaret Değişimi: Negatiften Pozitife, Alınması yada satılması gereken tarih(Sabah): 2022-12-14, Açılış Değeri: 144.1999969482422, Bütçe: 50.20021057128906, Hisseler: 69.0
Tarih: 2022-12-14, İşaret Değişimi: Pozitiften Negatife, Alınması yada satılması gereken tarih(Sabah): 2022-12-15, Açılış Değeri: 140.60000610351562, Bütçe: 9751.600631713867, Hisseler: 0
Tarih: 2022-12-20, İşaret Değişimi: Negatiften Pozitife, Alınması yada satılması gereken tarih(Sabah): 2022-12-21, Açılış Değeri: 146.3000030517578, Bütçe: 95.80043029785156, Hisseler: 66.0
Tarih: 2022-12-23, İşaret Değişimi: Pozitiften Negatife, Alınması yada satılması gereken tarih(Sabah): 2022-12-24, Açılış Değeri: 143.3000030517578, Bütçe: 9553.600631713867, Hisseler: 0
Tarih: 2022-12-28, İşaret Değişimi: Negatiften Pozitife, Alınması yada satılması

In [ ]:

"""data["predictions"] = predictions
data["preds"] = preds
print(data[["percentage", "predictions", "preds"]])"""

In [9]:
import pickle

filename = './app/ml_models/test.pkl'
pickle.dump(xgb_clf, open(filename, 'wb'))

In [12]:
X.columns

Index(['RSI_14', 'STOCHk_9_6_3', 'STOCHd_9_6_3', 'stoch_diff',
       'STOCHRSIk_14_14_3_3', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
       'ADX_14', 'DMP_14', 'DMN_14', 'WILLR_14', 'CCI_14_0.015', 'ATRr_14',
       'hl_ratio_14', 'UO_7_14_28', 'ROC_14', 'Bull_power_13', 'Bear_power_13',
       'SMA_interpretation_5_Al', 'SMA_interpretation_5_Güçlü Al',
       'SMA_interpretation_5_Güçlü Sat', 'SMA_interpretation_5_Nötr',
       'SMA_interpretation_10_Al', 'SMA_interpretation_10_Güçlü Al',
       'SMA_interpretation_10_Güçlü Sat', 'SMA_interpretation_10_Nötr',
       'SMA_interpretation_20_Al', 'SMA_interpretation_20_Güçlü Al',
       'SMA_interpretation_20_Güçlü Sat', 'SMA_interpretation_20_Nötr',
       'SMA_interpretation_50_Al', 'SMA_interpretation_50_Güçlü Al',
       'SMA_interpretation_50_Güçlü Sat', 'SMA_interpretation_50_Nötr',
       'EMA_interpretation_5_Al', 'EMA_interpretation_5_Güçlü Al',
       'EMA_interpretation_5_Güçlü Sat', 'EMA_interpretation_5_Nötr',
    